<a href="https://colab.research.google.com/github/biditdas18/Generative_Adversarial_Network_Fake_Celebrity_Faces/blob/master/GAN_Generating_Fake_Faces.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Google Colab was unable to handle such huge dataset of images and crashed constantly, hence this section of soce was implemented on my loal machine to extract all the faces of the celebrities from 50000 images and same them as a compressed numpy file

In [0]:
cd /content/drive/My Drive/

/content/drive/My Drive


In [0]:
pwd

'/content/drive/My Drive'

In [0]:
!unzip celeba-dataset.zip && rm celeba-dataset.zip

unzip:  cannot find or open celeba-dataset.zip, celeba-dataset.zip.zip or celeba-dataset.zip.ZIP.


In [0]:
pip install mtcnn

     |████████████████████████████████| 2.3MB 3.5MB/s 


In [0]:
# Extract and resize the faces into a new dataset
from os import listdir
import numpy as np
from PIL import Image
from mtcnn.mtcnn import MTCNN
import matplotlib.pyplot as plt

# load an image as an RGB numpy array
def load_image(filename):
  # load image from the file
  image = Image.open(filename)
  # convert to RGB if needed
  image = image.convert('RGB')
  # conver to array
  pixels = np.asarray(image)
  return pixels

# Extract the face from the loaded image and resize
def extract_face(model,pixels, required_size=(80,80)):
  # detect face in the image
  faces = model.detect_faces(pixels)
  # skip cases where we could not detect a face
  if len(faces) == 0:
    return None
  # extract details of the face
  x1,y1,width,height = faces[0]['box']
  # make sure the cooridinates are positive
  x1,y1 = abs(x1),abs(y1)
  # obtain second set of coordinates for convinience
  x2,y2 = x1+width,y1+height
  # retrive face pixels
  face_pixels = pixels[y1:y2,x1:x2]
  # resize the pixels to the model size
  image = Image.fromarray(face_pixels)
  image = image.resize(required_size)
  face_array = np.asarray(image)
  return face_array

# load images and extract faces for all images in a directory
def load_faces(directory,n_faces):
  model = MTCNN()
  faces = []
  # enumerate files
  for filename in listdir(directory):
    #load the image
    pixels = load_image(directory + filename)
    # get face
    face = extract_face(model,pixels)
    if face is None:
      continue
    # store
    faces.append(face)
    print(len(faces), face.shape)
    # stop once we have enough
    if len(faces) >= n_faces:
      break
  return np.asarray(faces)

# directory that contains all the images
directory = 'img_align_celeba/'
# load and extract all faces
all_faces = load_faces(directory,50000)
print('Loaded:',all_faces.shape)
# save in compressed format
np.savez_compressed('img_align_celeba.npz', all_faces)

Using TensorFlow backend.


KeyboardInterrupt: ignored

**The development of the GAN section was done on Google colab inorder to get GPU support**

**Developing The GAN**

In [0]:
cd /content/drive/My Drive/Kaggle

/content/drive/My Drive/Kaggle


In [0]:
pwd

'/content/drive/My Drive/Kaggle'

In [0]:
# Make necessary imports
import tensorflow as tf
from tensorflow.keras.layers import Dense, Reshape, Flatten,\
Conv2D,Conv2DTranspose, LeakyReLU, Dropout
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam

import numpy as np
import matplotlib.pyplot as plt

In [0]:
# define standalone discriminator model
def define_discriminator(in_shape=(80,80,3)):
	model = Sequential()
	
	model.add(Conv2D(128, (5,5), padding='same', input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	
	model.add(Conv2D(128, (5,5), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	
	model.add(Conv2D(128, (5,5), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	
	model.add(Conv2D(128, (5,5), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	
	model.add(Conv2D(128, (5,5), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# classifier
	model.add(Flatten())
	model.add(Dropout(0.4))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

In [0]:
# define standalone generator model
def define_generator(latent_dim):
  model = Sequential()
  # foundation of 5 x 5 feature map
  n_nodes = 128 * 5 * 5
  model.add(Dense(n_nodes, input_dim=latent_dim))
  model.add(LeakyReLU(alpha=0.2))
  model.add(Reshape((5,5,128)))

  model.add(Conv2DTranspose(128,(4,4),strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2DTranspose(128,(4,4),strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2DTranspose(128,(4,4),strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2DTranspose(128,(4,4),strides=(2,2), padding='same'))
  model.add(LeakyReLU(alpha=0.2))

  model.add(Conv2D(3,(5,5),activation='tanh', padding='same'))
  return model

# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model,d_model):
  d_model.trainable = False
  model = Sequential()
  # add generator
  model.add(g_model)
  # add the discriminator
  model.add(d_model)
  # compile model
  opt = Adam(lr=0.0002, beta_1=0.5)
  model.compile(loss='binary_crossentropy',optimizer=opt)
  return model
  

In [0]:
# load and prepare training images
def load_real_samples():
  # load the face dataset
  data = np.load('img_align_celeba.npz')
  X = data['arr_0']
  # Convert from unsigned int to float
  X = X.astype('float32')
  # scale from [0,255] to [-1,1]
  X = (X - 127.5)/127.5
  return X

In [0]:
# Select Real sample
def generate_real_samples(dataset, n_samples):
  i =np.random.randint(0,dataset.shape[0],n_samples)
  # retrice the selected images
  X = dataset[i]
  # generate 'real' class labels (1)
  y = np.ones((n_samples,1))
  return X,y

In [0]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim,n_samples):
  # generate points in the latent space
  X_input = np.random.randn(latent_dim * n_samples)
  # reshape into batch of inputs for the network
  X_input = X_input.reshape(n_samples, latent_dim)
  return X_input


In [0]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
  # generate points in the latent space
  X_input = generate_latent_points(latent_dim,n_samples)
  # predicts outputs
  X = g_model.predict(X_input)
  # create fake class labels (0)
  y = np.zeros((n_samples,1))
  return X,y

In [0]:

# Create a folder to store generated images
import sys,os
if not os.path.exists('gan_images'):
  os.makedirs('gan_images')

# create and save a plot of generated images
def save_plot(examples,epoch,n=10):
  # scale from [-1,1] to [0,1]
  examples = (examples + 1)/2.0
  # plot images
  for i in range(n * n):
    plt.subplot(n,n,1+i)
    plt.axis('off')
    plt.imshow(examples[i])
  # save the plot
  filename = 'gan_images/generated_plot_e%03d.png'%(epoch+1)
  plt.savefig(filename)
  plt.close()

In [0]:
# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch,g_model,d_model,dataset,latent_dim,n_samples=100):
  # prepare real samples
  X_real,y_real = generate_real_samples(dataset, n_samples)
  # evaluate discriminator on real samples
  _,acc_real = d_model.evaluate(X_real,y_real, verbose=0)
  # prepare fake examples
  X_fake,y_fake = generate_fake_samples(g_model, latent_dim,n_samples)
  # evaluate discriminator on real samples
  _,acc_fake = d_model.evaluate(X_fake,y_fake, verbose=0)
  # summarize discriminator performance
  print('Accuracy Real: %.0f%%, Fake: %.f%%' %(acc_real*100,acc_fake*100))
  # save plot
  save_plot(X_fake,epoch)
  # save the generator model tile file
  filename = 'generator_model_%03d.h5'%(epoch+1)
  g_model.save(filename)
  

In [0]:

# train the generator and discriminator
def train(g_model,d_model,gan_model,dataset,latent_dim,n_epochs=100, n_batch=128):
  batch_per_epoch = int(dataset.shape[0]/n_batch)
  half_batch = int(n_batch/2)
  # manually enumerate epochs
  for i in range(n_epochs):
    # enumerate batches over the training set
    for j in range(batch_per_epoch):
      #get randomly selected real images
      X_real,y_real = generate_real_samples(dataset,half_batch)
      # update discriminator model weights
      d_loss1,_= d_model.train_on_batch(X_real,y_real)
      # generate fake samples
      X_fake,y_fake = generate_fake_samples(g_model,latent_dim,half_batch)
      # update discriminator model weights
      d_loss2,_ = d_model.train_on_batch(X_fake,y_fake)
      # prepare points on the latent space as inputs for the generator
      X_gan = generate_latent_points(latent_dim,n_batch)
      # create inverted labels for the fake samples
      y_gan = np.ones((n_batch,1))
      # update generator through discriminators's error
      g_loss = gan_model.train_on_batch(X_gan,y_gan)
      #summarize loss on this batch
      print('>%d, %d/%d, d1=%.3f, d2=%.3f g=%.3f' %
				(i+1, j+1, batch_per_epoch, d_loss1, d_loss2, g_loss))
    # evaluate the model performance
    if (i+1)%10 == 0:
      summarize_performance(i,g_model,d_model,dataset,latent_dim)

In [0]:
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create gan
gan_model = define_gan(g_model,d_model)
# load image data
dataset = load_real_samples()
# train model 
train(g_model, d_model, gan_model, dataset, latent_dim)

Streaming output truncated to the last 5000 lines.
>88, 74/390, d1=0.465, d2=0.173 g=2.809
>88, 75/390, d1=0.510, d2=0.394 g=2.778
>88, 76/390, d1=0.225, d2=0.262 g=3.345
>88, 77/390, d1=0.487, d2=0.274 g=3.187
>88, 78/390, d1=0.286, d2=0.201 g=3.496
>88, 79/390, d1=0.415, d2=0.243 g=3.017
>88, 80/390, d1=0.413, d2=0.300 g=2.761
>88, 81/390, d1=0.256, d2=0.320 g=3.474
>88, 82/390, d1=0.458, d2=0.138 g=2.744
>88, 83/390, d1=0.181, d2=0.296 g=3.294
>88, 84/390, d1=0.287, d2=0.251 g=3.471
>88, 85/390, d1=0.201, d2=0.151 g=3.472
>88, 86/390, d1=0.265, d2=0.260 g=2.898
>88, 87/390, d1=0.277, d2=0.395 g=3.322
>88, 88/390, d1=0.316, d2=0.295 g=3.239
>88, 89/390, d1=0.494, d2=0.279 g=2.831
>88, 90/390, d1=0.238, d2=0.226 g=2.856
>88, 91/390, d1=0.322, d2=0.393 g=3.316
>88, 92/390, d1=0.304, d2=0.238 g=2.799
>88, 93/390, d1=0.398, d2=0.418 g=3.550
>88, 94/390, d1=0.335, d2=0.223 g=3.142
>88, 95/390, d1=0.280, d2=0.261 g=3.571
>88, 96/390, d1=0.457, d2=0.186 g=3.084
>88, 97/390, d1=0.260, d2=0.3